In [2]:
!pip install tensorflow==2.15.0 --ignore-installed


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow==2.15.0 from https://files.pythonhosted.org/packages/93/c0/a774286d0383419f558deb27096e5de9f9facd6c27df8e9f9af6fba2f77e/tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)

In [3]:
!pip install tensorflow==2.15.0 numpy==1.26.4 matplotlib==3.8.2 scikit-learn==1.4.2 seaborn==0.13.2 pandas==2.2.1 scipy==1.12.0


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import os
import zipfile

In [6]:
# Define the zip file name and the directory to extract to
zip_path = "american-sign-language-letters-subdir.zip"  # replace with your actual zip filename
extract_to = "data"  # or any folder name you want

import os
print(os.getcwd())

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted '{zip_path}' to '{extract_to}/'")

/sfs/gpfs/tardis/home/mwm6nq/DS4002
Extracted 'american-sign-language-letters-subdir.zip' to 'data/'


In [9]:
# Load raw images and split, better practice to clean after split
data_dir = "data/subdir"
img_height, img_width = 64, 64
batch_size = 32

# Train/validation split from train_val folder
train_val_dir = "data/train_val"
test_dir = "data/test"

# Training set
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

# Validation set
raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

# Test set — loaded from separate folder
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int"
)

NotFoundError: Could not find directory data/train_val

In [8]:
# Preprocessing function: grayscale + normalize scales image of pixels
def preprocess(image, label):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
# Apply preprocessing to train and test ds
train_ds = raw_train_ds.map(preprocess)
val_ds = raw_val_ds.map(preprocess)